# Cheat Sheet: Clustering con K-Means y Metodos Jerarquicos

Este cuaderno amplia el ejercicio original brindando contexto, explicaciones paso a paso y comentarios para entender cada decision en un flujo de clustering no supervisado.

## Indice Razonado
- [0. Contexto](#0-contexto)
- [1. Preparacion del entorno](#1-preparacion-del-entorno)
  - [1.1. Librerias clave](#11-librerias-clave)
  - [1.2. Configuracion de advertencias](#12-configuracion-de-advertencias)
- [2. Generacion de datos sintenticos](#2-generacion-de-datos-sintenticos)
  - [2.1. Dataset para K-Means](#21-dataset-para-k-means)
  - [2.2. Visualizacion inicial](#22-visualizacion-inicial)
- [3. K-Means paso a paso](#3-k-means-paso-a-paso)
  - [3.1. Metodo del codo](#31-metodo-del-codo)
  - [3.2. Coeficiente de silueta](#32-coeficiente-de-silueta)
- [4. Clustering jerarquico con primer dataset](#4-clustering-jerarquico-con-primer-dataset)
  - [4.1. Dendrograma complete + euclidiana](#41-dendrograma-complete--euclidiana)
  - [4.2. Dendrograma complete + manhattan](#42-dendrograma-complete--manhattan)
  - [4.3. Dendrograma single + euclidiana](#43-dendrograma-single--euclidiana)
- [5. Segundo dataset jerarquico](#5-segundo-dataset-jerarquico)
  - [5.1. Generacion y visualizacion](#51-generacion-y-visualizacion)
  - [5.2. Complete linkage con seleccion por silueta](#52-complete-linkage-con-seleccion-por-silueta)
  - [5.3. Single linkage con seleccion por silueta](#53-single-linkage-con-seleccion-por-silueta)
- [6. Conclusiones y siguientes pasos](#6-conclusiones-y-siguientes-pasos)

## 0. Contexto
Trabajamos con datos sintéticos para practicar clustering. El objetivo es entender cómo elegir el número de grupos con K-Means y cómo interpretar distintas configuraciones de clustering jerárquico, comparando resultados con métricas como el método del codo y el coeficiente de silueta.

## 1. Preparacion del entorno

### 1.1. Librerias clave
Importamos librerias para generar datos, aplicar clustering y visualizar resultados.

In [ ]:
import numpy as np
import warnings

import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage

### 1.2. Configuracion de advertencias
Reducimos ruido en la salida para concentrarnos en los resultados relevantes.

In [ ]:
warnings.filterwarnings("ignore")
SEED = 1234
np.random.seed(SEED)

print(f"Advertencias suprimidas. Semilla global fijada en {SEED}.")

## 2. Generacion de datos sintenticos

### 2.1. Dataset para K-Means
Creamos cuatro grupos compactos alrededor de dos centroides principales para facilitar la interpretacion visual del clustering.

In [ ]:
# Generamos cuatro nubes de puntos alrededor de dos coordenadas principales
X1 = np.random.normal(loc=5, scale=1, size=(10, 1))
X2 = np.random.normal(loc=5, scale=1, size=(10, 1))
X3 = np.random.normal(loc=20, scale=1, size=(10, 1))
X4 = np.random.normal(loc=20, scale=1, size=(10, 1))

Y1 = np.random.normal(loc=5, scale=1, size=(10, 1))
Y2 = np.random.normal(loc=20, scale=1, size=(10, 1))
Y3 = np.random.normal(loc=5, scale=1, size=(10, 1))
Y4 = np.random.normal(loc=20, scale=1, size=(10, 1))

XX = np.concatenate((X1, X2, X3, X4), axis=0)
YY = np.concatenate((Y1, Y2, Y3, Y4), axis=0)

X_kmeans = np.concatenate((XX, YY), axis=1)
print(f"Dataset K-Means: {X_kmeans.shape[0]} puntos, {X_kmeans.shape[1]} dimensiones")

### 2.2. Visualizacion inicial
Grafica de dispersión para confirmar la estructura de grupos antes de aplicar clustering.

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(X_kmeans[:, 0], X_kmeans[:, 1], s=30, alpha=0.7)
plt.xlabel('Caracteristica 1')
plt.ylabel('Caracteristica 2')
plt.title('Distribucion original de los datos')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

## 3. K-Means paso a paso
Analizamos cómo seleccionar el número adecuado de clusters mediante dos heurísticas comunes.

### 3.1. Metodo del codo
Calculamos la inercia (suma de distancias cuadraticas a los centroides) para distintos `k` y buscamos el punto donde la mejora se estabiliza.

In [ ]:
max_clusters = 10
k_values = range(1, max_clusters + 1)
inertia = []

for k in k_values:
    # Cada ejecucion re-estima centroides y acumula la inercia resultante
    km = KMeans(n_clusters=k, n_init="auto", random_state=SEED)
    km.fit(X_kmeans)
    inertia.append(km.inertia_)

plt.figure(figsize=(7, 4))
plt.plot(k_values, inertia, marker='o')
plt.xticks(k_values)
plt.xlabel('Numero de clusters (k)')
plt.ylabel('Inercia (SSE)')
plt.title('Metodo del codo con K-Means')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

### 3.2. Coeficiente de silueta
Evaluamos la cohesion y separacion de los clusters midiendo el promedio del coeficiente de silueta para `k \geq 2`.

In [ ]:
silhouette_scores = []

for k in range(2, max_clusters + 1):
    km = KMeans(n_clusters=k, n_init="auto", random_state=SEED)
    labels = km.fit_predict(X_kmeans)
    score = silhouette_score(X_kmeans, labels)
    silhouette_scores.append(score)

best_k_silhouette = np.argmax(silhouette_scores) + 2  # desplazamiento por comenzar en k=2

plt.figure(figsize=(7, 4))
plt.plot(range(2, max_clusters + 1), silhouette_scores, marker='o', color='tab:orange')
plt.xticks(range(2, max_clusters + 1))
plt.xlabel('Numero de clusters (k)')
plt.ylabel('Coeficiente medio de silueta')
plt.title('Silueta promedio por k')
plt.axvline(best_k_silhouette, color='gray', linestyle='--', alpha=0.6)
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

print(f"Mejor k segun silueta: {best_k_silhouette}")

**Nota:** El metodo del codo sugiere identificar el punto donde la inercia deja de reducirse drásticamente, mientras que la silueta cuantifica la compacidad y separacion. Con ambos criterios suele seleccionarse un `k` alrededor de 4 para este dataset.

## 4. Clustering jerarquico con primer dataset
Comparamos distintos criterios de enlace aplicando dendrogramas sobre el mismo conjunto inicial.

### 4.1. Dendrograma complete + euclidiana
El enlace `complete` evalua la distancia maxima entre puntos de dos clusters. Usamos distancia euclidiana como medida basica.

In [ ]:
linkage_complete_eu = linkage(X_kmeans, method='complete', metric='euclidean')
plt.figure(figsize=(8, 4))
dendrogram(linkage_complete_eu)
plt.title('Dendrograma (complete, euclidiana)')
plt.xlabel('Observaciones')
plt.ylabel('Distancia')
plt.show()

### 4.2. Dendrograma complete + manhattan
La distancia Manhattan (cityblock) suma diferencias absolutas y puede ser mas robusta ante outliers en ciertas direcciones.

In [ ]:
linkage_complete_manhattan = linkage(X_kmeans, method='complete', metric='cityblock')
plt.figure(figsize=(8, 4))
dendrogram(linkage_complete_manhattan)
plt.title('Dendrograma (complete, Manhattan)')
plt.xlabel('Observaciones')
plt.ylabel('Distancia')
plt.show()

### 4.3. Dendrograma single + euclidiana
El enlace `single` considera la minima distancia entre clusters, produciendo cadenas cuando hay puntos cercanos aislados.

In [ ]:
linkage_single_eu = linkage(X_kmeans, method='single', metric='euclidean')
plt.figure(figsize=(8, 4))
dendrogram(linkage_single_eu)
plt.title('Dendrograma (single, euclidiana)')
plt.xlabel('Observaciones')
plt.ylabel('Distancia')
plt.show()

**Observacion:** Los dendrogramas muestran la altura a la que se fusionan los grupos. Con enlace `single` se observa el efecto de *chaining*, mientras que `complete` mantiene separaciones mas equilibradas.

## 5. Segundo dataset jerarquico
Generamos un conjunto mas complejo para observar como los metodos jerarquicos capturan estructuras alargadas.

### 5.1. Generacion y visualizacion
Combinamos una nube lineal con dos grupos compactos para desafiar los esquemas jerarquicos.

In [ ]:
X_line = np.random.normal(loc=12.5, scale=3, size=(40, 1))
Y_line = X_line * 1 + np.random.normal(loc=0, scale=1, size=(40, 1))

X_blob1 = np.random.normal(loc=5, scale=1, size=(10, 1))
Y_blob1 = np.random.normal(loc=20, scale=1, size=(10, 1))

X_blob2 = np.random.normal(loc=20, scale=1, size=(10, 1))
Y_blob2 = np.random.normal(loc=5, scale=1, size=(10, 1))

XX2 = np.concatenate((X_line, X_blob1, X_blob2), axis=0)
YY2 = np.concatenate((Y_line, Y_blob1, Y_blob2), axis=0)

X_hier = np.concatenate((XX2, YY2), axis=1)
print(f"Dataset jerarquico: {X_hier.shape[0]} puntos")

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(X_hier[:, 0], X_hier[:, 1], s=30, alpha=0.7)
plt.xlabel('Caracteristica 1')
plt.ylabel('Caracteristica 2')
plt.title('Segundo dataset: mezcla de patrones')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

### 5.2. Complete linkage con seleccion por silueta
Probamos distintos valores de `k`, evaluamos la silueta y visualizamos el mejor agrupamiento segun ese criterio.

In [ ]:
linkage_complete_hier = linkage(X_hier, method='complete', metric='euclidean')
plt.figure(figsize=(8, 4))
dendrogram(linkage_complete_hier)
plt.title('Dendrograma (complete, euclidiana) - Dataset 2')
plt.xlabel('Observaciones')
plt.ylabel('Distancia')
plt.show()

In [ ]:
range_k = range(2, 11)
silhouette_complete = []

for k in range_k:
    model = AgglomerativeClustering(
        n_clusters=k,
        metric='euclidean',
        linkage='complete'
    )
    labels = model.fit_predict(X_hier)
    silhouette_complete.append(silhouette_score(X_hier, labels, metric='euclidean'))

best_k_complete = range_k[np.argmax(silhouette_complete)]

plt.figure(figsize=(7, 4))
plt.plot(range_k, silhouette_complete, marker='o')
plt.title('Silueta promedio (complete linkage)')
plt.xlabel('Numero de clusters (k)')
plt.ylabel('Puntuacion de silueta')
plt.axvline(best_k_complete, color='gray', linestyle='--', alpha=0.6)
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

print(f'Mejor k para complete linkage: {best_k_complete}')

In [ ]:
model_complete = AgglomerativeClustering(
    n_clusters=best_k_complete,
    metric='euclidean',
    linkage='complete'
)
labels_complete = model_complete.fit_predict(X_hier)

plt.figure(figsize=(6, 6))
plt.scatter(X_hier[:, 0], X_hier[:, 1], c=labels_complete, cmap='viridis', s=40, alpha=0.8)
plt.xlabel('Caracteristica 1')
plt.ylabel('Caracteristica 2')
plt.title(f'Clustering jerarquico (complete) con k={best_k_complete}')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

### 5.3. Single linkage con seleccion por silueta
Repetimos el analisis con enlace `single`, que tiende a formar clusters alargados al priorizar la menor distancia entre grupos.

In [ ]:
linkage_single_hier = linkage(X_hier, method='single', metric='euclidean')
plt.figure(figsize=(8, 4))
dendrogram(linkage_single_hier)
plt.title('Dendrograma (single, euclidiana) - Dataset 2')
plt.xlabel('Observaciones')
plt.ylabel('Distancia')
plt.show()

In [ ]:
silhouette_single = []

for k in range_k:
    model = AgglomerativeClustering(
        n_clusters=k,
        metric='euclidean',
        linkage='single'
    )
    labels = model.fit_predict(X_hier)
    silhouette_single.append(silhouette_score(X_hier, labels, metric='euclidean'))

best_k_single = range_k[np.argmax(silhouette_single)]

plt.figure(figsize=(7, 4))
plt.plot(range_k, silhouette_single, marker='o', color='tab:green')
plt.title('Silueta promedio (single linkage)')
plt.xlabel('Numero de clusters (k)')
plt.ylabel('Puntuacion de silueta')
plt.axvline(best_k_single, color='gray', linestyle='--', alpha=0.6)
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

print(f'Mejor k para single linkage: {best_k_single}')

In [ ]:
model_single = AgglomerativeClustering(
    n_clusters=best_k_single,
    metric='euclidean',
    linkage='single'
)
labels_single = model_single.fit_predict(X_hier)

plt.figure(figsize=(6, 6))
plt.scatter(X_hier[:, 0], X_hier[:, 1], c=labels_single, cmap='plasma', s=40, alpha=0.8)
plt.xlabel('Caracteristica 1')
plt.ylabel('Caracteristica 2')
plt.title(f'Clustering jerarquico (single) con k={best_k_single}')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

## 6. Conclusiones y siguientes pasos
- K-Means requiere explorar varios `k`; combinar el metodo del codo con la silueta ofrece una decision mas robusta.
- Los dendrogramas ayudan a entender la estructura jerarquica, pero el criterio de corte (altura) debe apoyarse en metricas cuantitativas.
- El enlace `complete` suele equilibrar clusters, mientras que `single` es sensible a puntos cercanos y puede formar cadenas.
- Probar diferentes metricas de distancia o estandarizar variables es recomendable cuando las escalas difieren.
- Como ejercicio adicional, comparar estos resultados con `DBSCAN` o `GaussianMixture` para datasets con formas no esfericas.